In [ ]:
import h5py
from os.path import join
from mssa_learning.models.lstm_classifier import LSTMClassifier
from mssa_learning.tools.h5_loader import *
from mssa_learning.tools.transformations import *
import json
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from sklearn import preprocessing
import progressbar

### Meta parameters

In [ ]:
window_size = 30
hidden_size = 128
batch_size = 200
epochs = 10
num_layers = 1
scale = False
nb_components = None

In [ ]:
nb_labels = None

In [ ]:
data_folder = join("..", "data", "dataset_converted" , "h5")

In [ ]:
dataset = "kalman"

In [ ]:
save_folder = "../results/" + dataset

### Load dataset

In [ ]:
transform = Compose([ToTensor()])

In [ ]:
loader = H5Loader(join(data_folder, dataset + ".h5"), window_size=window_size,
                  nb_components=nb_components, nb_labels=nb_labels, transform=transform)

In [ ]:
train_data, test_data = loader.extract_training_base()

### LSTM Classification

In [ ]:
D_in = loader.nb_components  # Dimension of the feature vector at each time step
D_out = loader.nb_labels  # number of classes to learn

In [ ]:
model = LSTMClassifier(D_in, hidden_size, D_out, window_size,
                       num_layers=num_layers, batch_size=batch_size,
                       epochs=epochs, scale=scale, save_folder=save_folder)

In [ ]:
model.fit(train_data, test_data)

In [ ]:
scalers = []
for d in range(loader.nb_components):
    scalers.append(preprocessing.MinMaxScaler())

In [ ]:
for d in range(loader.nb_components):
    input_data = []
    bar = progressbar.ProgressBar()
    for data in bar(train_data.dataset):
        input_data.append(data[0][d].numpy())
    scalers[d] = scalers[d].fit(input_data)